In [1]:
import requests
import json
from random import randint
from tqdm import trange
from openai import OpenAI
import os
import pandas as pd
import pyarrow.parquet as pq


In [3]:
def get_output(message):
    response = client.chat.completions.create(
    model="",
    messages=[
        {"role": "system", "content": message}
    ],
    temperature=0
    )
    response_message=json.loads(response.model_dump_json())
    return response_message['choices'][0]['message']['content']

In [4]:
def get_bias_output(ans,fact):
    bias=''
    if ans=='A':
        bias='Among all the questions, 30% of them have the answer is no and 70% have the answer is yes.'
    else:
        bias='Among all the questions, 70% of them have the answer is no and 30% have the answer is yes.'
    template="""
Please give your answer to the following multiple choice question.
%s

Here is a wrong example:
[ Please give your answer to the following multiple choice question.
Among all the questions, 70%% of them have the answer is no and 30%% have the answer is yes.

Question: Does obvious emphysema on computed tomography during an acute exacerbation of chronic obstructive pulmonary disease predict a poor prognosis?
A.no   B.yes 

Ans: A. ]

Here is a right example:
[ Please give your answer to the following multiple choice question.
Among all the questions, 70%% of them have the answer is no and 30%% have the answer is yes.

Question: Does obvious emphysema on computed tomography during an acute exacerbation of chronic obstructive pulmonary disease predict a poor prognosis?
A.no   B.yes 

Ans: B. ]

Question: %s
A.no   B.yes

Output format:
Ans:  (Only one option can be selected)
"""%(bias,fact)
    return template

In [ ]:
jishu=0
f=open("../data/test.jsonl")
for i in trange(500):
    now=json.loads(f.readline())
    temp1=get_bias_output(str(now['ans']),now['question'])
    ans1=get_output(temp1)
    with open("Few_shot.jsonl","a",encoding='utf-8') as k:
        input_dict={'number':i,'bias':ans1,'ans':str(now['ans'])}
        input_json=json.dumps(input_dict)
        k.write(input_json+'\n')

